## Introduction to built-in datasources

Read from CSV and create temp view

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F
sc = SparkContext("local", "Internal read/write")
spark = (SparkSession
         .builder
         .appName('Internal read/write')
         .getOrCreate())

In [2]:
csv_file = '../datasets/departuredelays.csv'
df = (spark
      .read
      .format('csv')
      .option('inferSchema', 'true')
      .option('header', 'true')
      .load(csv_file))
df.createOrReplaceTempView("us_delay_flights_tbl")

Perform some queries on temp table:

* using Spark SQL

In [3]:
spark.sql("""
SELECT origin, destination, distance
FROM us_delay_flights_tbl
WHERE distance > 1000
ORDER BY distance DESC""").show(10)

spark.sql("""
SELECT date, origin, destination, distance
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER BY delay DESC""").show(10)

+------+-----------+--------+
|origin|destination|distance|
+------+-----------+--------+
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
|   HNL|        JFK|    4330|
+------+-----------+--------+
only showing top 10 rows

+-------+------+-----------+--------+
|   date|origin|destination|distance|
+-------+------+-----------+--------+
|2190925|   SFO|        ORD|    1604|
|1031755|   SFO|        ORD|    1604|
|1022330|   SFO|        ORD|    1604|
|1051205|   SFO|        ORD|    1604|
|1190925|   SFO|        ORD|    1604|
|2171115|   SFO|        ORD|    1604|
|1071040|   SFO|        ORD|    1604|
|1051550|   SFO|        ORD|    1604|
|3120730|   SFO|        ORD|    1604|
|1261104|   SFO|        ORD|    1604|
+-------+------+-----------+--------+
only showing top 10 ro

In [4]:
spark.sql("""
SELECT date, origin, destination, distance,
CASE
  WHEN delay > 360 THEN 'Very Long Delays'
  WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
  WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
  WHEN delay > 0 and delay < 60  THEN  'Tolerable Delays'
  WHEN delay = 0 THEN 'No Delays'
  ELSE 'Early'
END AS Flight_Delays
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER BY delay DESC""").show(10)


+-------+------+-----------+--------+----------------+
|   date|origin|destination|distance|   Flight_Delays|
+-------+------+-----------+--------+----------------+
|2190925|   SFO|        ORD|    1604|Very Long Delays|
|1031755|   SFO|        ORD|    1604|Very Long Delays|
|1022330|   SFO|        ORD|    1604|     Long Delays|
|1051205|   SFO|        ORD|    1604|     Long Delays|
|1190925|   SFO|        ORD|    1604|     Long Delays|
|2171115|   SFO|        ORD|    1604|     Long Delays|
|1071040|   SFO|        ORD|    1604|     Long Delays|
|1051550|   SFO|        ORD|    1604|     Long Delays|
|3120730|   SFO|        ORD|    1604|     Long Delays|
|1261104|   SFO|        ORD|    1604|     Long Delays|
+-------+------+-----------+--------+----------------+
only showing top 10 rows



* using Spark DataFrame API

In [5]:
(df
 .select('origin', 'destination', 'distance')
 .where(F.col('distance') > 1000)
 .orderBy(F.col('delay').desc())
 .show(10))

(df
 .select('date', 'origin', 'destination', 'distance')
 .where((F.col('delay') > 120)
        & (F.col('origin')=='SFO')
        & (F.col('destination')=='ORD'))
 .orderBy(F.col('delay').desc())
 .show(10))


+------+-----------+--------+
|origin|destination|distance|
+------+-----------+--------+
|   SFO|        ORD|    1604|
|   PDX|        DFW|    1404|
|   EGE|        JFK|    1517|
|   ONT|        DFW|    1033|
|   CMH|        LAX|    1734|
|   AUS|        LAX|    1079|
|   IAD|        DFW|    1018|
|   LAS|        MIA|    1889|
|   FAT|        DFW|    1141|
|   BOS|        MIA|    1093|
+------+-----------+--------+
only showing top 10 rows

+-------+------+-----------+--------+
|   date|origin|destination|distance|
+-------+------+-----------+--------+
|2190925|   SFO|        ORD|    1604|
|1031755|   SFO|        ORD|    1604|
|1022330|   SFO|        ORD|    1604|
|1051205|   SFO|        ORD|    1604|
|1190925|   SFO|        ORD|    1604|
|2171115|   SFO|        ORD|    1604|
|1071040|   SFO|        ORD|    1604|
|1051550|   SFO|        ORD|    1604|
|3120730|   SFO|        ORD|    1604|
|1261104|   SFO|        ORD|    1604|
+-------+------+-----------+--------+
only showing top 10 ro

In [6]:
from pyspark.sql.functions import udf, pandas_udf
import pandas as pd
import time

def delay_classification(delay):
    if delay > 360:
        return 'Very Long Delays'
    if 120 < delay < 360:
        return 'Long Delays'
    if 60 < delay < 120:
        return 'Short Delays'
    if 0 < delay < 60:
        return 'Tolerable Delays'
    if delay == 0:
        return 'No Delays'
    return 'Early'

@pandas_udf('string')
def pudf_delay_classification(delay: pd.Series) -> pd.Series:
    return delay.apply(delay_classification)

udf_delay_classification = udf(delay_classification)

start = time.perf_counter()
(df
 .select('date', 'origin', 'destination', 'distance',
         udf_delay_classification(F.col('delay')).alias('Flight_Delays'))
 .where((F.col('delay') > 120)
        & (F.col('origin')=='SFO')
        & (F.col('destination')=='ORD'))
 .orderBy(F.col('delay').desc())
 .show(10))
print('udf: ', time.perf_counter() - start, ' seconds')

start = time.perf_counter()
(df
 .select('date', 'origin', 'destination', 'distance',
         pudf_delay_classification(F.col('delay')).alias('Flight_Delays'))
 .where((F.col('delay') > 120)
        & (F.col('origin')=='SFO')
        & (F.col('destination')=='ORD'))
 .orderBy(F.col('delay').desc())
 .show(10))
print('pandas_udf: ', time.perf_counter() - start, ' seconds')


+-------+------+-----------+--------+----------------+
|   date|origin|destination|distance|   Flight_Delays|
+-------+------+-----------+--------+----------------+
|2190925|   SFO|        ORD|    1604|Very Long Delays|
|1031755|   SFO|        ORD|    1604|Very Long Delays|
|1022330|   SFO|        ORD|    1604|     Long Delays|
|1051205|   SFO|        ORD|    1604|     Long Delays|
|1190925|   SFO|        ORD|    1604|     Long Delays|
|2171115|   SFO|        ORD|    1604|     Long Delays|
|1071040|   SFO|        ORD|    1604|     Long Delays|
|1051550|   SFO|        ORD|    1604|     Long Delays|
|3120730|   SFO|        ORD|    1604|     Long Delays|
|1261104|   SFO|        ORD|    1604|     Long Delays|
+-------+------+-----------+--------+----------------+
only showing top 10 rows

udf:  2.428698637  seconds
+-------+------+-----------+--------+----------------+
|   date|origin|destination|distance|   Flight_Delays|
+-------+------+-----------+--------+----------------+
|2190925|   

### Managed and unmanaged tables

Managed table


In [7]:
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema=schema, header=True)
flights_df.write.saveAsTable(
    "managed_us_delay_flights_tbl",
    mode='ignore'
)
flights_df.show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [8]:
spark.sql("""
SELECT *
FROM managed_us_delay_flights_tbl
LIMIT 10
""")

DataFrame[date: string, delay: int, distance: int, origin: string, destination: string]

Unmanaged table

In [9]:
# noinspection SqlNoDataSourceInspection
spark.sql(f"""
CREATE TABLE IF NOT EXISTS us_delay_flights_tbl(date STRING, delay INT,
distance INT, origin STRING, destination STRING)
USING csv OPTIONS (PATH
'{csv_file}')
""")


DataFrame[]

In [10]:
spark.sql("""
SELECT *
FROM us_delay_flights_tbl
LIMIT 10
""").show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+



In [11]:
# or equivalently
(flights_df
 .write
 #.option("path", "/tmp/us_flights_delay")
 .saveAsTable("us_delay_flights_tbl",
              mode='overwrite',
              path='/tmp/us_flights_delay'))

### Creating views
Global temp view

In [12]:
spark.sql("""
CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
SELECT date, delay, origin, destination
FROM us_delay_flights_tbl
WHERE origin = 'SFO'
""")

spark.sql("""
SELECT *
FROM global_temp.us_origin_airport_SFO_global_tmp_view
LIMIT 10
""").show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1011250|   55|   SFO|        JFK|
|1012230|    0|   SFO|        JFK|
|1010705|   -7|   SFO|        JFK|
|1010620|   -3|   SFO|        MIA|
|1010915|   -3|   SFO|        LAX|
+-------+-----+------+-----------+
only showing top 5 rows



In [13]:
# or equivalently
df_sfo = spark.sql("""
SELECT date, delay, origin, destination
FROM us_delay_flights_tbl WHERE origin = 'SFO'""")
try:
    df_sfo.createGlobalTempView('us_origin_airport_SFO_global_tmp_view')
except:
    print('Table already existed')

Table already existed


Local (session-scoped) temp view

In [14]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW us_origin_airport_SFO_tmp_view AS
SELECT date, delay, origin, destination
FROM us_delay_flights_tbl
WHERE origin = 'SFO'
""")

spark.sql("""
SELECT *
FROM us_origin_airport_SFO_tmp_view
LIMIT 10
""").show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1011250|   55|   SFO|        JFK|
|1012230|    0|   SFO|        JFK|
|1010705|   -7|   SFO|        JFK|
|1010620|   -3|   SFO|        MIA|
|1010915|   -3|   SFO|        LAX|
+-------+-----+------+-----------+
only showing top 5 rows



In [15]:
# or equivalently
df_sfo = spark.sql("""
SELECT date, delay, origin, destination
FROM us_delay_flights_tbl WHERE origin = 'SFO'""")
try:
    df_sfo.createTempView('us_origin_airport_SFO_tmp_view')
except:
    print('Table already existed')


Table already existed


In [16]:
spark.catalog.listDatabases()#%%


[Database(name='default', description='default database', locationUri='file:/Users/lap01195/Documents/random_code/spark/git/notebook/spark-warehouse')]

In [17]:
spark.catalog.listTables()

[Table(name='managed_us_delay_flights_tbl', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='us_origin_airport_sfo_tmp_view', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [18]:
spark.catalog.listColumns("us_delay_flights_tbl")

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]